# Movie Recommendation System

In [1]:
import pandas as pd

## Read movie data

In [2]:
df = pd.read_csv('movies.csv')
df.head()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
df.shape

(62423, 3)

## Clean Title

In [4]:
import re #regular expression

def clean_title(title):
    return re.sub("[^a-zA-Z0-9 ]", "", title) #remove special character

In [5]:
df['clean_title'] = df['title'].apply(clean_title)
df.head()

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2)) #change title into number
tfidf = vectorizer.fit_transform(df['clean_title'])

In [7]:
#search tclean_title
from  sklearn.metrics.pairwise import cosine_similarity
import numpy as np

#change search title into vector and find similarity
def search(title):
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:] #indices of most similar 5 movies
    results = df.iloc[indices][::-1]
    return results

## Building interactive widget

In [8]:
import ipywidgets as widgets
from IPython.display import display

movie_input = widgets.Text(
    value = "Toy Story",
    description = "Movie Title:",
    disable = False
)
movie_list = widgets.Output() 
def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data['new']
        if len(title)>5:
            display(search(title))
            
movie_input.observe(on_type, names = 'value')

display(movie_input, movie_list)

Text(value='Toy Story', description='Movie Title:')

Output()

## find movie similar to our movie

In [9]:
rating = pd.read_csv('ratings.csv') 
rating = rating.drop(['timestamp'],axis=1)
rating

,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5
...,...,...,...
25000090,162541,50872,4.5
25000091,162541,55768,2.5
25000092,162541,56176,2.0
25000093,162541,58559,4.0


In [10]:
rating.dtypes

userId       int64
movieId      int64
rating     float64
dtype: object

## Finding users who like the same movie

In [11]:
movie_id = 1

In [12]:
similar_users = rating[(rating['movieId'] == movie_id) & (rating['rating'] > 4)]['userId'].unique()

In [13]:
similar_users

array([    36,     75,     86, ..., 162527, 162530, 162533], dtype=int64)

In [14]:
similar_user_recs = rating[(rating['userId'].isin(similar_users)) & (rating['rating'] > 4)]['movieId']

In [15]:
similar_user_recs

5101            1
5105           34
5111          110
5114          150
5127          260
            ...  
24998854    60069
24998861    67997
24998876    78499
24998884    81591
24998888    88129
Name: movieId, Length: 1358326, dtype: int64

In [16]:
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

similar_user_recs = similar_user_recs[similar_user_recs > 0.1] #recommend movie which 10% of similar user liked


In [17]:
print(similar_user_recs)

movieId
1        1.000000
318      0.445607
260      0.403770
356      0.370215
296      0.367295
           ...   
953      0.103053
551      0.101195
1222     0.100876
745      0.100345
48780    0.100186
Name: count, Length: 113, dtype: float64


## Finding how much all users like movies

In [18]:
all_users = rating[(rating['movieId'].isin(similar_user_recs.index) & (rating['rating'] > 4))]

In [19]:
all_users

,userId,movieId,rating
0,1,296,5.0
29,1,4973,4.5
48,1,7361,5.0
72,2,110,5.0
76,2,260,5.0
...,...,...,...
25000062,162541,5618,4.5
25000065,162541,5952,5.0
25000078,162541,7153,5.0
25000081,162541,7361,4.5


In [20]:
all_users_recs = all_users['movieId'].value_counts() / len(all_users['userId'].unique())

In [21]:
all_users_recs

movieId
318      0.342220
296      0.284674
2571     0.244033
356      0.235266
593      0.225909
           ...   
551      0.040918
50872    0.039111
745      0.037031
78499    0.035131
2355     0.025091
Name: count, Length: 113, dtype: float64

## Cresting recommendation score

In [22]:
rec_percentages = pd.concat([similar_user_recs, all_users_recs], axis = 1) 
rec_percentages.columns = ['similar','all']

In [23]:
rec_percentages

,similar,all
movieId,,
1,1.000000,0.124728
318,0.445607,0.342220
260,0.403770,0.222207
356,0.370215,0.235266
296,0.367295,0.284674
...,...,...
953,0.103053,0.045792
551,0.101195,0.040918
1222,0.100876,0.066877


In [24]:
rec_percentages['score'] = rec_percentages['similar'] / rec_percentages['all']

In [25]:
rec_percentages = rec_percentages.sort_values('score',ascending=False)

In [26]:
rec_percentages

,similar,all,score
movieId,,,
1,1.000000,0.124728,8.017414
3114,0.280648,0.053706,5.225654
2355,0.110539,0.025091,4.405452
78499,0.152960,0.035131,4.354038
4886,0.235147,0.070811,3.320783
...,...,...,...
2858,0.216724,0.167634,1.292845
296,0.367295,0.284674,1.290232
79132,0.166817,0.131384,1.269693


In [27]:
rec_percentages.head(10).merge(df, left_index = True, right_on = 'movieId')

,similar,all,score,movieId,title,genres,clean_title
0,1.000000,0.124728,8.017414,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
3021,0.280648,0.053706,5.225654,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 2 1999
2264,0.110539,0.025091,4.405452,2355,"Bug's Life, A (1998)",Adventure|Animation|Children|Comedy,Bugs Life A 1998
14813,0.152960,0.035131,4.354038,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX,Toy Story 3 2010
4780,0.235147,0.070811,3.320783,4886,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy,Monsters Inc 2001
580,0.216618,0.067513,3.208539,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical,Aladdin 1992
6258,0.228139,0.072268,3.156862,6377,Finding Nemo (2003),Adventure|Animation|Children|Comedy,Finding Nemo 2003
587,0.179400,0.059977,2.991150,595,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX,Beauty and the Beast 1991
8246,0.203504,0.068453,2.972889,8961,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,Incredibles The 2004
359,0.253411,0.085764,2.954762,364,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,Lion King The 1994


## Building Recommendation function

In [28]:
def find_similar_movies(movie_id):
    similar_users = rating[(rating['movieId'] == movie_id) & (rating['rating'] > 4)]['userId'].unique()
    similar_user_recs = rating[(rating['userId'].isin(similar_users)) & (rating['rating'] > 4)]['movieId']

    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
    similar_user_recs = similar_user_recs[similar_user_recs > 0.1] #recommend movie which 10% of similar user liked

    all_users = rating[(rating['movieId'].isin(similar_user_recs.index) & (rating['rating'] > 4))]
    all_users_recs = all_users['movieId'].value_counts() / len(all_users['userId'].unique())

    rec_percentages = pd.concat([similar_user_recs, all_users_recs], axis = 1) 
    rec_percentages.columns = ['similar','all']

    rec_percentages['score'] = rec_percentages['similar'] / rec_percentages['all']
    rec_percentages = rec_percentages.sort_values('score',ascending=False)
    return rec_percentages.head(10).merge(df, left_index = True, right_on = 'movieId')[['score','title','genres']]

## Creating interactive recommendation widget

In [30]:
movie_name_input = widgets.Text(
    value = 'Toy Story', 
    description = 'Movie Title:', 
    disable = False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data['new']
        if len(title) > 4:
            results = search(title)
            movie_id = results.iloc[0]['movieId']
            display(find_similar_movies(movie_id)) 

movie_name_input.observe(on_type, names = 'value')

display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()